In [ ]:
%load_ext autoreload
%autoreload 2
%config Completer.use_jedi = True

- we compare the distance measure for the same time series 

In [ ]:
import matplotlib as mpl
from matplotlib import pyplot as plt
mpl.rc("figure", dpi=100)
import numpy as np
import jack
from lmz import *


In [ ]:
# Data preparation
import cellsaw
import cellsaw.io_utils
import notebookhelper

loaders = [jack.loads5 , jack.load509, jack.load1290]
loaders += [jack.getmousecortex, jack.loadwater, jack.pancreatic, jack.loadcereb]

labels = ['Assigned_subcluster','Cluster','New_cellType','labels','label','labels','celltype','labels']

# loading data-> [[[anndata]]] 
#datasets = [ [load(subsample=1000, seed = seed) for load in loaders] for seed in [31337,42,69,420,1312]]
datasets = [ load(subsample=1000, seed = 45)[-4:] for load in loaders]

In [ ]:
results = []
import scanorama
from sklearn.semi_supervised import LabelPropagation
from sklearn import metrics
if False:
    for i,(dataset,label) in enumerate(zip(datasets,labels)):
        scanorama.integrate_scanpy(dataset)
        y = dataset[-1].obs[label]
        src = dataset[-1].obsm['X_scanorama']
        diffusor = LabelPropagation(gamma = .75).fit(src,y)

        target = dataset[:-1]
        predicted = diffusor.predict( np.vstack([a.obsm['X_scanorama'] for a in target]) )
        size  = [ x.X.shape[0] for x in target]
        pred = np.split(predicted,np.add.accumulate(size))
        results+=[{'method':'scanorama','dataset':i, 'slice':j, 'score':metrics.adjusted_rand_score(t.obs[label],p) } 
                      for j,(t,p) in enumerate(zip(target,pred))]

    
    
    
    

In [ ]:
np.add.accumulate([4,4,5,6])

In [ ]:
from sklearn.semi_supervised import LabelPropagation
from sklearn import metrics
from cellsaw.merge import diffusion
import functools
from cellsaw.merge import Merge
from cellsaw import preprocess as prep
if False:
    for i,(dataset,label) in enumerate(zip(datasets,labels)):
        target = dataset[:-1]
        # PCA
        prep.annotate_genescore(dataset,selector = 'cell_ranger')
        m = Merge(dataset,pca=50) 

        # prepare kernel
        targetsizes  = [ x.X.shape[0] for x in target]
        krnl = functools.partial( diffusion.linear_assignment_kernel_XXX, tsizes = targetsizes)

        # fit predict
        diffusor = diffusion.Diffusion(n_neighbors_intra=5, 
                                     n_neighbors_inter= 4,
                                     sigmafac = 64,
                                     linear_assignment_factor=5,
                                     kernel = krnl)
        y = dataset[-1].obs[label]
        diffusor.fit(m.projections[1][-1],y)
        predicted = diffusor.predict( np.vstack(m.projections[1][:-1]) )

        # split precition
        pred = np.split(predicted,np.add.accumulate(targetsizes))
        results+=[{'method':'Ours','dataset':i, 'slice':j, 'score':metrics.adjusted_rand_score(t.obs[label],p) } 
                      for j,(t,p) in enumerate(zip(target,pred))]

    

In [ ]:
from sklearn.semi_supervised import LabelPropagation
from sklearn import metrics
from cellsaw.merge import diffusion
import functools
from cellsaw.merge import Merge
from cellsaw import preprocess as prep
from ubergauss import tools
from lmz import *
    
def mkscr(i):
    dataset = datasets[i]
    label = labels[i]
    
    target = dataset[:-1]
    target.reverse()
    # PCA
    prep.annotate_genescore(dataset,selector = 'cell_ranger')
    m = Merge(dataset,pca=50) 
    
    # prepare kernel
    targetsizes  = [ x.X.shape[0] for x in target]
    krnl = functools.partial( diffusion.linear_assignment_kernel_XXX, tsizes = targetsizes)
    
    # fit predict
    diffusor = diffusion.Diffusion(n_neighbors_intra=5, 
                                 n_neighbors_inter= 4,
                                 sigmafac = 64,
                                 linear_assignment_factor=5,
                                 kernel = krnl)
    y = dataset[-1].obs[label]
    diffusor.fit(m.projections[1][-1],y)
    predicted = diffusor.predict( np.vstack(m.projections[1][:-1]) )
    
    # split precition
    pred = np.split(predicted,np.add.accumulate(targetsizes))
    return [{'method':'Ours','dataset':i, 'slice':j, 'score':metrics.adjusted_rand_score(t.obs[label],p) } 
                  for j,(t,p) in enumerate(zip(target,pred))]


#zz = tools.xmap(mkscr,Range(datasets))
#zzz = [z for zzz in zz for z in zzz]

In [ ]:
import pandas as pd
import seaborn as sns
df = pd.DataFrame(zzz+results)
sns.boxplot(data = df, x='score', y= 'method')
sns.heatmap(df.pivot_table('score','method','dataset'),square=True) 

In [ ]:
for e in datasets:
    print(len(e))

In [ ]:
from sklearn.semi_supervised import LabelPropagation
from sklearn import metrics
from cellsaw.merge import diffusion
import functools
from cellsaw.merge import Merge
from cellsaw import preprocess as prep
from ubergauss import tools
from lmz import *

from hyperopt import fmin, tpe, hp, Trials
from hyperopt.pyll import scope



def mkscr(xxx):
    i, par = xxx
    pid, na , nr ,sig, lsa,umap = par['pid'],par['na'],par['nr'],par['sig'],par['lsa'],par['umap']
    dataset = [d.copy() for d in datasets[i]]
    dataset.reverse()
    label = labels[i]
    target = dataset[1:]
    
    # PCA
    prep.annotate_genescore(dataset,selector = 'cell_ranger')
    m = Merge(dataset,pca=50,umaps = [umap]) 
    
    # prepare kernel
    targetsizes  = [ x.X.shape[0] for x in target]
    krnl = functools.partial( diffusion.linear_assignment_kernel_XXX, tsizes = targetsizes)
    
    # fit predict
    diffusor = diffusion.Diffusion(n_neighbors_intra=na, 
                                 n_neighbors_inter= nr,
                                 sigmafac = sig,
                                 linear_assignment_factor=lsa,
                                 kernel = krnl)
    y = dataset[0].obs[label]
    diffusor.fit(m.projections[pid][0],y)
    predicted = diffusor.predict( np.vstack(m.projections[pid][1:]) )
    # split precition
    pred = np.split(predicted,np.add.accumulate(targetsizes))
    res =  [{'method':'Ours','dataset':i, 'slice':j, 'score':metrics.adjusted_rand_score(t.obs[label],p) }  for j,(t,p) in enumerate(zip(target,pred))]
    return np.mean([r['score'] for r in res]) 



def evalparams(params):
    print(params)
    return -np.mean(tools.xmap(mkscr, ( (i,params) for i in Range(datasets)) ))

def optimize():
    #pid, na , nr ,sig, lsa,umap = par['pid'],par['na'],par['nr'],par['sig'],par['lsa'],par['umap']
    space = { 
        'umap' :          scope.int(hp.quniform('umap',5,15,1)),
        'lsa':hp.uniform('lsa',.001,10),
        'sig':hp.uniform('sig',.001,100),
        'na' :          scope.int(hp.quniform('na',1,10,1)),
        'nr' :          scope.int(hp.quniform('nr',1,10,1)),
        'pid' :          scope.int(hp.quniform('pid',1,2,1))
    } #hp.uniform('linear_assignment_factor',4,12)}

    trials = Trials()
    best = fmin(fn=evalparams,
                space = space,
        algo=tpe.suggest,
        trials = trials,
        max_evals=100)

    print(best)
    return trials

t = optimize()